In [9]:
import pandas as pd

from experiment.utils.tables.reports_raw_table import ReportsRawTable
from experiment.utils import dbutils
from experiment.utils import transformation

In [10]:
db = dbutils.DatabaseUtils()

In [11]:
# read the csv file
raw_ct_brain = transformation.get_project_root() / "tmp" / "data" / "raw_ct_brain.csv"
df = pd.read_csv(raw_ct_brain)

In [12]:
reports_raw, Base = ReportsRawTable()
# Base.metadata.create_all(db.engine)

data_to_insert = []
cols_to_upsert = ["patient_no", "full_name", "report_original"]
for index, row in df.iterrows():
    data_to_insert.append(
        {
            "report_id": index + 1, # use df index as report_id
            "protocol_no": row["PROTOKOL_NO"],
            "patient_no": row["HASTA_NO"],
            "full_name": transformation.hash_value(row["AD_SOYAD"]),
            "report_original": row["ACIKLAMA"],
        }
    )

db.upsert_values(reports_raw, data_to_insert, cols_to_upsert, ["report_id"])